# Xarray to Zarr on GCS with CamHD Data Example
Here we build an Xarray Dataset from Dask delayed functions, and save the images to a GCS zarr group using xarray's to_zarr() functionality. Processing takes place on the workers and memory pressure is very low because all the images do not need to be loaded at the same time.

#### Imports

In [ ]:
%matplotlib inline
import pycamhd.pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

#### Create a list of frames to analyze using the dbcamhd.json database

In [ ]:
dbcamhd = pd.read_json('dbcamhd.json', orient="records", lines=True)
dbcamhd.tail()

In [ ]:
fileindex = 2064
filename = dbcamhd.filename[fileindex]
frame_count = dbcamhd.frame_count[fileindex]
n_images = 100
frame_numbers = np.linspace(750,frame_count-6000, n_images, dtype=np.int64())
filename

#### Create timestamps for frames

In [ ]:
from datetime import datetime

In [ ]:
timestamps = []
for i in range(len(frame_numbers)):
    timestamps.append(datetime.fromtimestamp(dbcamhd.timestamp[fileindex] + frame_numbers[i]/29.95))

In [ ]:
timestamps[0:5]

#### Create Xarray Dataset out of Dask delayed functions

In [ ]:
from dask import delayed, compute
import dask.array as da

In [ ]:
delayed_frames = []
moov_atom = camhd.get_moov_atom(filename)
for frame_number in frame_numbers:
    delayed_frames.append(da.from_delayed(
                            delayed(camhd.get_frame)(filename, frame_number, 'rgb24', moov_atom),
                            shape=(1080, 1920, 3), dtype=np.uint8)[None,:,:,:])

In [ ]:
delayed_frames[0]

In [ ]:
all_data = da.concatenate(delayed_frames, axis=0)
all_data

In [ ]:
ds = xr.DataArray(all_data, dims=['time', 'y', 'x', 'channel'],
                  coords={'time': timestamps}
                 ).to_dataset(name='video')
ds

#### Start a KubeCluster

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=16)
cluster

In [ ]:
from dask import delayed, compute
from dask.distributed import Client
client = Client(cluster)
client

#### Use to_zarr() to save to a GCS bucket

In [ ]:
import gcsfs
pid = 'PROJECTID' # your pangeo project id
fs_orig = gcsfs.GCSFileSystem(project=pid, token='browser')

In [ ]:
fs = gcsfs.GCSFileSystem(project=pid, token=fs_orig.session.credentials)
gcsmap = gcsfs.mapping.GCSMap('test-bucket-01', gcs=fs, check=True, create=True)

In [ ]:
ds.to_zarr(gcsmap)